In [2]:
import json

import pandas as pd
import pickle
import numpy as np
import plotly.express as px

In [3]:
df_op = pd.read_csv("/Users/nikolaushouben/Desktop/WattCast/results/mpc_results/1_county/Los_Angeles/RandomForest_operations.csv", index_col=0)

px.line(df_op)

In [12]:
df_op = pd.read_csv("/Users/nikolaushouben/Desktop/WattCast/results/mpc_results/1_county/Los_Angeles/RandomForest_operations.csv", index_col=0)

px.line(df_op)